In [1]:
import pandas as pd
import numpy as np
from warnings import filterwarnings

In [2]:
filterwarnings("ignore")
airbnb_raw = pd.read_csv('https://media.githubusercontent.com/media/imadahmad97/EDA-of-Airbnb-Data/main/DataSets/Raw%20Dataset%20from%20kaggle/Listings.csv', encoding='iso-8859-1')
airbnb_review_raw = pd.read_csv('https://media.githubusercontent.com/media/imadahmad97/EDA-of-Airbnb-Data/main/DataSets/Raw%20Dataset%20from%20kaggle/Reviews.csv')
display(airbnb_raw.head())
display(airbnb_review_raw.head())

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,minimum_nights,maximum_nights,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
1,3705183,39 mÃÂ² Paris (Sacre CÃ âur),10328771,2013-11-29,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",NaN,NaN,NaN,f,1.0,...,2,1125,100.0,10.0,10.0,10.0,10.0,10.0,10.0,f


,listing_id,review_id,date,reviewer_id
0,11798,330265172,2018-09-30,11863072
1,15383,330103585,2018-09-30,39147453
2,16455,329985788,2018-09-30,1125378
3,17919,330016899,2018-09-30,172717984
4,26827,329995638,2018-09-30,17542859


In [3]:
# Correcting datatype for string columns (note: Pandas stores strings as objects)
airbnb_raw[['name', 'host_location', 'host_response_time', 
            'neighbourhood', 'district', 'city', 
            'property_type', 'room_type']] = airbnb_raw[['name','host_location', 
            'host_response_time','neighbourhood','district',
            'city','property_type','room_type']].astype('str')

# Correcting labelling and datatype for boolean columns
airbnb_raw['host_is_superhost'] = airbnb_raw['host_is_superhost'].map({'t':True,'f':False}).astype(bool)
airbnb_raw['host_has_profile_pic'] = airbnb_raw['host_has_profile_pic'].map({'t':True,'f':False}).astype(bool)
airbnb_raw['host_identity_verified'] = airbnb_raw['host_identity_verified'].map({'t':True,'f':False}).astype(bool)
airbnb_raw['instant_bookable'] = airbnb_raw['instant_bookable'].map({'t':True,'f':False}).astype(bool)

# Creating 2 columns, one with host_since in DateTime format, and one with the year values of host_since in DateTime format
airbnb_raw['host_since_dt'] = pd.to_datetime(airbnb_raw['host_since'])
airbnb_raw['host_since_dt_year'] = airbnb_raw['host_since_dt'].apply(lambda x: str(x.year))

In [4]:
airbnb_review_raw['review_date_dt'] = pd.to_datetime(airbnb_review_raw['date'])
airbnb_review_raw['days_since_last_review'] = (airbnb_review_raw.sort_values('review_date_dt').
                                             groupby('listing_id').review_date_dt.shift() - airbnb_review_raw.
                                             review_date_dt).dt.days.abs()

In [5]:
airbnb_review_raw['year_month'] = airbnb_review_raw['review_date_dt'].dt.strftime('%Y_%m')
airbnb_year_month = airbnb_review_raw.groupby(['listing_id','year_month'])['review_id'].count()
airbnb_year_month = airbnb_year_month.unstack(level=-1).fillna(0)

In [6]:
# These columns needed to be inserted at a specific location or at the front
airbnb_year_month.insert(3,'2009_03',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(1,'2008_12',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(0,'2008_10',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(0,'2008_09',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(0,'2008_08',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(0,'2008_07',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(0,'2008_06',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(0,'2008_05',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(0,'2008_04',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(0,'2008_03',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(0,'2008_02',[*[0] * len(airbnb_year_month)])
airbnb_year_month.insert(0,'2008_01',[*[0] * len(airbnb_year_month)])

# These columns could be named without using .insert, as we are adding them to the right side
airbnb_year_month['2021_04'] = 0
airbnb_year_month['2021_05'] = 0
airbnb_year_month['2021_06'] = 0
airbnb_year_month['2021_07'] = 0
airbnb_year_month['2021_08'] = 0
airbnb_year_month['2021_09'] = 0
airbnb_year_month['2021_10'] = 0
airbnb_year_month['2021_11'] = 0
airbnb_year_month['2021_12'] = 0

In [7]:
# Using the .agg function, we were able to add all the columns in one block of code
airbnb_review_info = airbnb_review_raw.groupby(['listing_id']).agg(
    review_date_min=('review_date_dt', np.min),
    review_date_max=('review_date_dt', np.max),
    review_id_distinct_count=('review_id',lambda x: x.nunique()),
    reviewer_id_distinct_count=('reviewer_id',lambda x: x.nunique()),
    avg_no_of_day_btw_review=('days_since_last_review', np.nanmean), #ignore nan in calculating avg
).reset_index()

In [8]:
airbnb_raw_plus_review = airbnb_raw.merge(airbnb_review_info, on='listing_id', how='left')
airbnb_raw_plus_review = airbnb_raw_plus_review.merge(airbnb_year_month, on='listing_id', how='left')
airbnb_raw_plus_review = airbnb_raw_plus_review.replace('nan', 'No Data') # Some columns had nan as a string

In [9]:
exchange_table = [['Bangkok','THB', 37.89], ['Cape Town', 'ZAR', 17.92], ['Hong Kong', 'HKD',7.85], ['Istanbul', 'TRY',18.41], 
                  ['Mexico City', 'MXN',20.17], ['New York', 'USD',1], ['Paris', 'EUR',1.04], ['Rio de Janeiro', 'BRL',5.39], 
                  ['Rome', 'EUR',1.04], ['Sydney', 'AUD',1.54]]
exchange_table_df = pd.DataFrame(exchange_table, columns=['city', 'currency','currency_rate'])
airbnb_raw_plus_review = airbnb_raw_plus_review.merge(exchange_table_df, on='city', how='left')
airbnb_raw_plus_review['price_USD']=airbnb_raw_plus_review['price']/airbnb_raw_plus_review['currency_rate']

In [10]:
airbnb_raw_plus_review.head()

,listing_id,name,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_total_listings_count,...,2021_06,2021_07,2021_08,2021_09,2021_10,2021_11,2021_12,currency,currency_rate,price_USD
0,281420,"Beautiful Flat in le Village Montmartre, Paris",1466919,2011-12-03,"Paris, Ile-de-France, France",No Data,NaN,NaN,False,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EUR,1.04,50.961538
1,3705183,39 mÃÂ² Paris (Sacre CÃ âur),10328771,2013-11-29,"Paris, Ile-de-France, France",No Data,NaN,NaN,False,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EUR,1.04,115.384615
2,4082273,"Lovely apartment with Terrace, 60m2",19252768,2014-07-31,"Paris, Ile-de-France, France",No Data,NaN,NaN,False,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EUR,1.04,85.576923
3,4797344,Cosy studio (close to Eiffel tower),10668311,2013-12-17,"Paris, Ile-de-France, France",No Data,NaN,NaN,False,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EUR,1.04,55.769231
4,4823489,Close to Eiffel Tower - Beautiful flat : 2 rooms,24837558,2014-12-14,"Paris, Ile-de-France, France",No Data,NaN,NaN,False,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,EUR,1.04,57.692308
